# Задача 13. Метод Ньютона
*Выполнил:  Кириллов Максим 3821Б1ПР1*

*Метод Ньютона* нахождения корня уравнения $f(x) = 0$ заключается в итерациях вида
$
x_{i+1} = x_i - \frac{f(x_i)}{f'(x_i)}.
$
Написать функцию `mynewton(f, df, [x0, x1])`, реализующую метод Ньютона,
где 
`f` – строка, задающая правую часть $f(x)$ уравнения,
`df` – строка, задающая  $f'(x)$,
`[x0, x1]` – отрезок локализации.
Функция должна возвращать найденный корень с макимально возможной точностью.

Написать программу, тестирующую эту
функцию и сравнивающую ее с `scipy.optimize.newton`, `scipy.optimize.fsolve` на уравнениях:

$x^3 - 2x - 5 = 0, \qquad 0\le x \le 3$ (исторический пример Валлиса),

$\sin x = 0, \qquad 1 \le x \le 4,$

$x^3  = 0.001, \qquad -1 \le x \le 1,$

$\ln x + \frac{2}{3} = 0, \qquad 0 \le x \le 1,$

$\mathop{\rm sgn} (x-2)\, \sqrt{|x-2|} = 0, \qquad 1 \le x \le 4,$

$ \arctan x = \frac{\pi}{3}, \qquad 0 \le x \le 5,$

$\frac{1}{x - \pi} = 0, \qquad 0 \le x \le 5.$

Программа должна печатать таблицу, в которой указываются найденные функциями `mynewton`,
`scipy.optimize.newton`, `scipy.optimize.fsolve` решения, их относительные ошибки, и количества затраченных итераций.
Сравнить и сделать выводы.

## Решение
Импортируем необходимые библиотеки


In [547]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import pandas as pd

Запишем функции и их производные. Дополнительно запишем отрезки локализации

$f_1(x) = x^3 - 2x - 5$

In [548]:
x1 = [0, 3]
def f1(x):
    return x ** 3 - 2 * x - 5

def df1(x):
    return 3 * x ** 2 - 2

$f_2(x) = \sin x$

In [549]:
x2 = [1, 4]
def f2(x):
    return np.sin(x)

def df2(x):
    return np.cos(x)

$f_3(x) = x^3 - 0.001$

In [550]:
x3 = [-1, 1]
def f3(x):
    return x ** 3 - 0.001

def df3(x):
    return 3 * x ** 2

$f_4(x) = \ln x + \frac{2}{3}$

In [551]:
x4 = [0, 1]
def f4(x):
    return np.log(x) + 2 / 3

def df4(x):
    return 1 / x

$f_5(x) = \mathop{\rm sgn} (x-2)\, \sqrt{|x-2|}$

In [552]:
x5 = [1, 4]
def f5(x):
    return np.sign(x - 2) * np.sqrt(abs(x - 2))

def df5(x):
    return ((x - 2) * np.sign(x - 2)) / (2 * (abs(x - 2) ** (3 / 2)))

$f_6(x) = \arctan x - \frac{\pi}{3}$

In [553]:
x6 = [0, 5]
def f6(x):
    return np.arctan(x) - np.pi / 3

def df6(x):
    return 1 / (1 + x ** 2)

$f_7(x) = \frac{1}{x - \pi}$

In [554]:
x7 = [0, 5]
def f7(x):
    return 1 / (x - np.pi)

def df7(x):
    return -1 / ((x - np.pi) ** 2)

Запишем функцию, реализующую метод Ньютона

In [555]:
eps = 1e-14      #  задание точности
def mynewton(f, df, x0, x1):
    count = 0     
    x = (x0 + x1) / 2       # возьмём середину заданного отрезка, как начальную точку
    if (x == 0): # исключение ошибки ZeroDivisionError
        x = x0  
    while (abs(f(x)) > eps): # находим корень, пока он больше заданной точности
        x = x - f(x) / df(x)
        count += 1
        if (count > 100000): # ограничение по итерациям
            x, count
            break
    return x, count

Найдём решения для заданных функций

Для этого создадим массивы, в которых будем запоминать решения функций. Отдельно запишем массивы функций и промежутков локализации, чтобы было удобно обращаться к ним в дальнейшем


In [556]:
mynewton_roots = []
newton_roots = []
fsolve_roots = []
operations_count = []
func_array = [f1, f2, f3, f4, f5, f6, f7]
deriv_array = [df1, df2, df3, df4, df5, df6, df7]
x_ranges = [x1, x2, x3, x4, x5, x6, x7]

Затем найдём решения при помощи прохождения в цикле по всем функциям

In [557]:
for i in range(7):
    mynewton_roots.append(mynewton(func_array[i], deriv_array[i], x_ranges[i][0], x_ranges[i][1])[0])
    operations_count.append(mynewton(func_array[i], deriv_array[i], x_ranges[i][0], x_ranges[i][1])[1])
    # нахождение начальной точки
    x = (x_ranges[i][0] + x_ranges[i][1]) / 2
    if (x == 0): # исключение ошибки ZeroDivisionError
        x = x_ranges[i][0]
    newton_roots.append(optimize.newton(func_array[i], x, disp=False))
    fsolve_roots.append(optimize.fsolve(func_array[i], x)[0])

c:\Python310\lib\site-packages\scipy\optimize\_minpack_py.py:175: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


Найдём относительные ошибки у найденных решений между созданной функцией и функциями библиотеки scipy.

In [558]:
newton_err = []
fsolve_err = []
for i in range(7):
    newton_err.append((newton_roots[i] - mynewton_roots[i]) * 100)
    fsolve_err.append((fsolve_roots[i] - mynewton_roots[i]) * 100)

Создадим таблицу и занесём в неё все необходимые значения

In [559]:
# список всех функций
func = ["x^3 - 2x - 5 = 0", "sin(x) = 0", "x^3 = 0.001", "ln(x) + 2/3 = 0", "sgn(x-2)*sqrt(|x-2|) = 0", "arctan(x) = pi/3", "1/(x-pi) = 0"]
table = pd.DataFrame({ 'mynewton': np.array(mynewton_roots), 
                     'optimize.newton': np.array(newton_roots),
                     'optimize.fsolve': np.array(fsolve_roots),
                     'Затраченные итерации': np.array(operations_count),
                     'Относительная ошибка newton': np.array(newton_err),
                     'Относительная ошибка fsolve': np.array(fsolve_err)}, index = func)
table

,mynewton,optimize.newton,optimize.fsolve,Затраченные итерации,Относительная ошибка newton,Относительная ошибка fsolve
x^3 - 2x - 5 = 0,2.094551e+00,2.094551e+00,2.094551e+00,6,0.000000e+00,0.000000e+00
sin(x) = 0,3.141593e+00,3.141593e+00,3.141593e+00,4,4.440892e-14,0.000000e+00
x^3 = 0.001,1.000000e-01,1.000000e-01,1.000000e-01,12,-1.498801e-13,-1.526557e-13
ln(x) + 2/3 = 0,5.134171e-01,5.134171e-01,5.134171e-01,3,1.110223e-14,0.000000e+00
sgn(x-2)*sqrt(|x-2|) = 0,1.500000e+00,2.000000e+00,2.000000e+00,100001,5.000000e+01,5.000000e+01
arctan(x) = pi/3,1.732051e+00,1.732051e+00,1.732051e+00,5,1.332268e-13,1.998401e-13
1/(x-pi) = 0,-1.805923e+14,-2.113417e+10,-1.129355e+83,48,1.805711e+16,-1.129355e+85


# Вывод
Проанализировав таблицу, можно сделать вывод, что относительные ошибки имеют достаточно низкое значение, что говорит о почти одинаковых результатах у созданной функции и функций библиотеки scipy. Однако в таблице есть функция, которая при большом количестве итераций использования реализованной функции метода Ньютона не меняет полученное решение, и мы не можем получить более точное значение. Также у последней функции довольно сильно различаются результаты как у созданной функции, так и у библиотечных функций.

Таким образом, метод Ньютона является довольно простым алгоритмом и обладает хорошей точностью вычисления, однако есть уравнения, для которых не получится получить точное решение.